## U-Net

In [21]:
from keras.layers import Input, Dense, Dropout, Flatten, concatenate, MaxPooling2D, Conv2D, Conv2DTranspose, UpSampling2D
from sklearn.cross_validation import train_test_split
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras import optimizers
from keras.optimizers import SGD
from keras.models import Model
K.set_image_dim_ordering('tf')
K.set_floatx('float32')

import pandas as pd
import numpy as np
np.random.seed(17)

train_data = np.load('train.npy')
train_target = np.load('train_target.npy')
train_data = preprocess_input(train_data)
x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,test_size=0.25, random_state=32)

In [29]:
inputs = Input((96, 96, 3))
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

dense = Flatten()(conv10)
dense = Dense(3, activation="softmax")(dense)

model = Model(inputs=[inputs], outputs=[dense])

# compile the model
sgd = SGD(lr=0.0001, momentum=0.9)
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [28]:
# print the model structure
for i, layer in enumerate(model.layers):
    print(i, layer.name)

(0, 'input_13')
(1, 'conv2d_126')
(2, 'conv2d_127')
(3, 'max_pooling2d_39')
(4, 'conv2d_128')
(5, 'conv2d_129')
(6, 'max_pooling2d_40')
(7, 'conv2d_130')
(8, 'conv2d_131')
(9, 'max_pooling2d_41')
(10, 'conv2d_132')
(11, 'conv2d_133')
(12, 'max_pooling2d_42')
(13, 'conv2d_134')
(14, 'conv2d_135')
(15, 'conv2d_transpose_19')
(16, 'concatenate_21')
(17, 'conv2d_136')
(18, 'conv2d_137')
(19, 'conv2d_transpose_20')
(20, 'concatenate_22')
(21, 'conv2d_138')
(22, 'conv2d_139')
(23, 'conv2d_transpose_21')
(24, 'concatenate_23')
(25, 'conv2d_140')
(26, 'conv2d_141')
(27, 'conv2d_transpose_22')
(28, 'concatenate_24')
(29, 'conv2d_142')
(30, 'conv2d_143')
(31, 'conv2d_144')


In [30]:
datagen = ImageDataGenerator(rotation_range=0.9, zoom_range=0.5, shear_range=0.3, \
                             horizontal_flip=False, vertical_flip=True, width_shift_range=0.5)
datagen.fit(train_data)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=15, shuffle=True), steps_per_epoch=len(x_train)/100, \
                    epochs=100, verbose=1, validation_data=(x_val_train, y_val_train))

Epoch 1/100
12/12 [==============================] - 7s - loss: 1.1488 - acc: 0.4722 - val_loss: 1.1672 - val_acc: 0.5061
Epoch 2/100
12/12 [==============================] - 3s - loss: 1.1370 - acc: 0.3722 - val_loss: 1.1814 - val_acc: 0.4842
Epoch 3/100
12/12 [==============================] - 3s - loss: 1.0906 - acc: 0.4611 - val_loss: 1.0215 - val_acc: 0.5182
Epoch 4/100
12/12 [==============================] - 3s - loss: 1.0155 - acc: 0.5222 - val_loss: 1.0195 - val_acc: 0.5231
Epoch 5/100
12/12 [==============================] - 3s - loss: 0.9920 - acc: 0.5611 - val_loss: 1.0140 - val_acc: 0.5255
Epoch 6/100
12/12 [==============================] - 3s - loss: 1.0309 - acc: 0.5056 - val_loss: 1.0136 - val_acc: 0.5158
Epoch 7/100
12/12 [==============================] - 3s - loss: 1.0268 - acc: 0.5140 - val_loss: 1.0106 - val_acc: 0.5158
Epoch 8/100
12/12 [==============================] - 3s - loss: 1.0257 - acc: 0.4556 - val_loss: 1.0135 - val_acc: 0.5061
Epoch 9/100
12/12 [=====

12/12 [==============================] - 3s - loss: 0.9391 - acc: 0.5889 - val_loss: 0.9885 - val_acc: 0.5182
Epoch 68/100
12/12 [==============================] - 3s - loss: 1.0283 - acc: 0.5056 - val_loss: 0.9875 - val_acc: 0.5182
Epoch 69/100
12/12 [==============================] - 3s - loss: 0.9525 - acc: 0.5222 - val_loss: 0.9875 - val_acc: 0.5255
Epoch 70/100
12/12 [==============================] - 3s - loss: 0.9326 - acc: 0.5699 - val_loss: 0.9997 - val_acc: 0.5280
Epoch 71/100
12/12 [==============================] - 3s - loss: 0.9883 - acc: 0.5278 - val_loss: 0.9997 - val_acc: 0.5207
Epoch 72/100
12/12 [==============================] - 3s - loss: 1.0423 - acc: 0.5056 - val_loss: 0.9926 - val_acc: 0.5328
Epoch 73/100
12/12 [==============================] - 3s - loss: 1.0083 - acc: 0.5278 - val_loss: 0.9912 - val_acc: 0.5280
Epoch 74/100
12/12 [==============================] - 3s - loss: 1.0154 - acc: 0.4778 - val_loss: 0.9893 - val_acc: 0.5255
Epoch 75/100
12/12 [=========